参考：
- <https://blog.csdn.net/weixin_42462202/article/details/99089020>
- <https://blog.csdn.net/pang_gua/article/details/86599782>
- <https://blog.csdn.net/leixiaohua1020/article/details/50534369>
- [网络抽象层单元类型 (NALU)](https://www.jianshu.com/p/a19f3e63b433)
- [I 帧,B帧,P帧,IDR帧的区别](https://blog.csdn.net/qq_29350001/article/details/73770702)
- [一步一步解析H.264码流的NALU(SPS,PSS,IDR)](guoh.org/lifelog/2013/10/h-264-bit-stream-sps-pps-idr-nalu/
)
- [LIVE555---testRTSPClient 实例](https://blog.csdn.net/qq_29350001/article/details/78051767)

# 为什么使用 RTP 协议？

- TCP 传输流媒体数据由于其可靠性，会造成很大的网络延时和卡顿。
- UDP 传输由于其不可靠性，会导致丢帧，如果是关键帧，则会花屏一个序列的时长。
- RTP 使用了 RTP 和 RTCP 两个子协议来完成。

  - RTP 使用 UDP 完成流媒体数据传输，保证其时效性。
  - RTCP 也是使用 UDP，但只传输控制信息，占带宽很小，此时上层根据 RTCP 的信息，按照其重要性决定是否对丢失的流媒体数据进行重传。
  - RTP 在 1025-65535 之间选择一个未使用的偶数端口号作为其端口，RTCP 则使用下一个奇数端口号作为其端口，进而组成一个 UDP 端口对，端口号 5004 和 5005 作为 RTP 和 RTCP 的默认端口号。

- RTP-Header 的流媒体特性，提供 帧边界、编码格式、序列号、时间戳等字段。
RTP 支持网络多播，而 TCP 不支持。

---

#  H.264的NALU格式

- 拆解：H264 --> 序列(SPS.PPS.IPBBP…) --> Frame(帧) --> slice(切片) --> 宏块 --> 子宏块。
- 序列：一段 H264 序列是指从一个 I 帧开始到下一个 I 帧前的所有帧的集合。

- NALU：H264 被封装在一个个 NALU（Network Abstraction Layer Unit）中进行传输。
NALU 以 [00 00 00 01] 为开始码，之后是 `NaluHeader`，再之后是 `NaluPayload`。

  eg: [00 00 00 01 67 2F A4 1E 23 59 1E 42 … ].

H.264原始码流（又称为“裸流”）是由一个一个的`NALU`组成的。他们的结构如下图所示。

![rtp](../assets/rtp10.jpg)

每个NALU之间使用`0x000001`（3Byte）或者`0x00000001`（4Byte）分隔开.

如果NALU对应的分片为一帧的开始就用`0x00000001`，否则就用`0x000001`。
H.264码流解析的步骤就是首先从码流中搜索`0x000001`和`0x00000001`，分离出NALU；然后再分析NALU的各个字段。

每个`NALU`的第一次字节都有特殊的含义，其内容如下:

|位    | 描述|
|------  |:---------:|
|bit[7]  | **最高位必须为0**|
|bit[5-6]|标记该NALU的**优先级(0, 1, 2, 3)**
bit[0-4]| **NALU单元的类型**|

常见的NALU单元类型：
```
00 00 00 01 67(SPS)
00 00 00 01 68(PPS)
00 00 00 01 65(IDR 帧)
00 00 00 01 61(P帧)
```

# H.264的三种RTP打包方式

RTP 封包时会将`[00 00 00 01]` 的开始码去除。(**注：在收到 RTP 包时需要在每个 NALU 头的位置拼接此开始码，并保存成h264文件即可播放)**

eg：[RTP-Header] + [ 67 2F A4 1E 23 59 1E 42 … ].

H.264可以由三种RTP封包策略:

- **单NALU打包(Single NALU)**

  一个RTP载荷包含一个完整的`NALU`,此时，RTP 的结构为:
  
  RtpHeader + RtpPayload(NaluHeader + NaluPayload).


- **聚合打包(STAP-A)**

  对于较小的NALU，一个RTP载荷可包含多个完整的`NALU`，通过`0x000001`或者`0x00000001`分隔。


- **分片打包(FU-A)**

  如果 NALU 长度超过 MTU(最大传输单元) 时，则需要对 NALU 进行分片封包。此时，RTP 的结构为 :
  
  RtpHeader + RtpPayload**(FuIndicator + FuHeader + NaluPayload).**
  
  会比完整包**多一个字节**的头信息，


**注意**：每一个RTP包都包含一个RTP头部和RTP荷载，这是固定的.比较常用的是`单NALU打包`和`分片打包`.


## 单NALU打包

所谓单NALU打包就是将一整个NALU的数据放入RTP包的载荷中
这是最简单的一种方式，无需过多的讲解

## 分片打包

每个RTP包都有大小限制的，因为RTP一般都是使用UDP发送，UDP没有流量控制，所以要限制每一次发送的大小，所以如果一个NALU的太大，就需要分成多个RTP包发送，如何分成多个RTP包，下面来好好讲一讲

首先要明确，RTP包的格式是绝不会变的，永远多是`RTP头+RTP载荷`

![rtp](../assets/rtp6.png)

RTP头部是固定的，那么只能在RTP载荷中去添加额外信息来说明这个RTP包是表示同一个NALU

如果是分片打包的话，那么在RTP载荷开始有两个字节的信息，然后再是NALU的内容

![rtp](../assets/rtp7.png)

- 第一个字节位FU Indicator，其格式如下

![rtp](../assets/rtp8.png)

    - 高三位：与NALU第一个字节的高三位相同
    - Type:28，表示该RTP包一个分片，为什么是28？因为H.264的规范中定义的，此外还有许多其他Type，这里不详讲

- 第二个字节位FU Header，其格式如下

![rtp](../assets/rtp9.png)

    - S：标记该分片打包的第一个RTP包
    - E：比较该分片打包的最后一个RTP包
    - Type：NALU的Type


发送分片的部分关键代码示意：
```c
// 取原始nalu的第一个第一个字节
naluType = frame[0];
// RTP包的载荷第一字节：将优先级设为3 （&0x60），然后将类型置为28
payload[0] = (naluType & 0x60) | 28;
//RTP包的载荷第二字节：的类型与NALU一致
payload[1] = naluType & 0x1F;
//除此之外，是起始分片时，第二字节最高位置为1：
payload[1] |= 0x80; // start
// 是结束分片时，第二字节第7位置为1：
rtpPacket->payload[1] |= 0x40; // end
```
## *结构体表示

### RtpHeader

```c
/**
 * RtpHeader，普遍占用12个字节
 * 
 * 由于 IP 协议采用大端序，这里需要转成小端序 (Java-Byte 是大端序，java 代码中可以不用转)，
 * 所以这里每一个字节内的各个属性跟标准 rtp 协议头刚好相反，
 * 并且在使用 "大于1bit" 的属性时需要将网络序转成字节序.
 */
typedef struct rtp_header_t
{
  	// 1byte (0)
  	unsigned int cc : 4;	  	/* CSRC count */
  	unsigned int x : 1;		 	/* header extension flag */
  	unsigned int p : 1;		  	/* padding flag */
  	unsigned int version : 2; 	/* protocol version */
  
  	// 1byte (1)
  	unsigned int pt : 7; 		/* payload type */
  	unsigned int m : 1;  		/* marker bit */
  	
  	// 2bytes (2,3)
  	unsigned int seq : 16; 		/* sequence number */
  
  	// 4bytes (4-7)
  	uint32_t ts;		   		/* timestamp */
  
  	// 4bytes (8-11)
  	uint32_t ssrc;		   		/* synchronization source */
  
  	// 4bytes csrc 可选位
  	// uint32_t csrc[1];        /* optional CSRC list */
};
```

- 属性 m: 表示是否到一帧的末尾。
  
    java 中可以使用 `isFrameEnd = buf[1] & 0x80 == 0x80 `来判断。
  `& 0x80`: 获取该 Byte 中第一个 bit 位. [ & 1000 0000 ]. `== 0x80`: 该 Byte 第一个 bit 值为1时，Byte值为 0x80 ([1000 0000]).

- 属性 seq:

  表示当前消息的序列号，在一定范围内每发出一个 RTP 包，seq 自增一次。
  
  java 中可以使用 `seq = (((int) buf[2] & 0xff) << 8) + ((int) buf[3] & 0xff)` 来获得其值。
  
### NaluHeader

```c
/**
 * NaluHeader，占用1个字节，在 RtpPayload 开头。
 * RTP 包含完整包时，RtpPayload = NaluHeader + NaluPayload.
 */
typedef struct nalu_header_t
{
  	unsigned int nalu_head_3 : 3;	/* 前三位填充 nalu-head 的前三位 */
  	unsigned int nalu_type : 5;		/* 后五位表示 nalu-type */
};
```
- nalu_type说明:

  表示 nal 单元的类型，1～12由H.264使用，24～31由其他应用使用，以下是具体定义。
  
```
	0     没有定义  
    1-23  NAL单元  单个 NAL 单元包
    1     不分区，非IDR图像的片
    2     片分区A
    3     片分区B
    4     片分区C
    5     IDR图像中的片
    6     补充增强信息单元（SEI）
    7     SPS
    8     PPS
    9     序列结束
    10    序列结束
    11    码流借宿
    12    填充
    13-23 保留  
    
    24    STAP-A   单一时间的组合包
    25    STAP-B   单一时间的组合包
    26    MTAP16   多个时间的组合包
    27    MTAP24   多个时间的组合包
    28    FU-A     分片的单元
    29    FU-B     分片的单元
    30-31 没有定义
```

### FuIndicator + FuHeader

```c
/**
 * FuIndicator + FuHeader. 各占一个字节，在 RtpHeader 之后。
 * NALU的长度超过 MTU 时，RTP 对其进行分片传输，称为 Fragmentation Unit(以下简称FU)。
 * RTP 包含切片包时，RtpPayload = FuIndicator + FuHeade + NaluPayload.
 */
typedef struct fu_indicator_header_t
{
  	unsigned int nalu_head_3 : 3;	/* FuIndicator, 前三位填充 nalu-header 的前三位 */
  	unsigned int nalu_type : 5;		/* FuIndicator, 后五位表示 nalu-type */
  	
  	unsigned int fu_flag : 3;		/* FuHeader, 前三位表示 fu 的标志位. (0x80/0x40/0x00) */
  	unsigned int nalu_head_5 : 5;	/* FuHeader, 后五位填充 nalu-header 的后五位. */
};
```

- 属性 nalu_type:

  占5个bit，用于确定是否为切片封装。

  **如果候选值在 1-12 之间，则表示未切片，当前属性属于 NaluHeader.**
  
  **如果候选值在 28-29 之间，则表示被切片，当前属性属于 FuIndicator，且后面会追加一个字节表示 FuHeader。**

  java 中可以使用 `isFU-A = buf[RTPHeader.len] & 0x1F == 0x1C` 判断是否属于切片包。
  
  `buf[RTPHeader.len]`: 取 RTP-Head 后的第一个 Byte( 即RTP-Payload开头）.
  
  `& 0x1F`: 获取后5个 bit 位。[ & 0001 1111 ]
  
  `== 0x1C`: FU-A 的值为 28


- 属性 fu_flag:

  **只有在确定为切片包时有效**，否则这个位置的字节已经属于 NaluPayload.
  
  占用3个bit， S | E | R，为 FU 的标志位，表示当前包为 FU 包的| 开头 | 结尾| 内容。

  S- StartFlag: 第1个 bit 位，为1表示 NALU-Start，则 flag 值为 0x80 ([1000 0000]);
  
  E- EndFlag: 第2个 bit 位，为1表示 NALU-End，则 flag 值为 0x40 ([0100 0000]);
  
  R- RemainFlag: 第3个 bit 位，保留位，**恒为0**.
  
  java 中可以使用 `Fu_Flag = buf[RTPHeader.len+1] & 0xE0` 获得当前 flag 值。

- **nalu_head 拼接**：

  **单NALU打包时为单独字节，分片打包时被切割封装在 FuIndicator 的前3位和 FuHeader的后5位。**
  
  可以通过 `nalu_header = (fu_indicator & 0xe0) | (fu_header & 0x1f)` 获得其值。
  **(候选值：0x67-SPS、0x68-PPS、0x66-SEI帧、0x65-IDR帧、0x61-P帧)**
 
   此时 nalu_header 的后五位仍然表示 nalu-type，但是取值范围在1-12之间。
   

不分片：
- 完整 Nalu 包需要写入 NaluStarter + NaluHeader + NaluPayload.

分片：
- Fu包为 Nalu的起始位置，需要写入 NaluStarter + NaluHeader + NaluPayload.
- Fu包为 Nalu的其他位置，只需要写入 NaluPayload.


```c
#define RTP_HEADER_LEN 12
#define NALU_STARTER_LEN 4 // [00 00 00 01] 的开始码
#define NALU_HEADER_LEN 1
#define FU_INDICATOR_LEN 1
#define FU_HEADER_LEN 1

int rtp_buffer_unpack(unsigned char *write_buf, int write_size)
{
   rtp_header_t *rtp_header = (rtp_header_t *)write_buf;
   unsigned char *rtp_payload = write_buf + RTP_HEADER_LEN; // 地址偏移
   uint8_t nalu_type = rtp_payload[0] & 0x1F; // NaluHeader的后5位 或 FuIndicator的后5位

   if (nalu_type == NALUType_FU_A) // 0x1C
   {
 		uint8_t fua_type = rtp_payload[1] & 0xE0; // FuHeader 的前三位
 		int header_len = RTP_HEADER_LEN + FU_INDICATOR_LEN + FU_HEADER_LEN;
 		unsigned char *nalu_payload = write_buf + header_len; // 地址偏移
 
 		if (fua_type == Fua_Start) // Fu包为 Nalu的起始位置，需要写入 NaluStarter + NaluHeader + NaluPayload.
 		{
   			uint8_t nalu_header = (rtp_payload[0] & 0xE0) | (rtp_payload[1] & 0x1F); // FuIndicator 的前3位和 FuHeader的后5位
   
   			int input_size = NALU_STARTER_LEN + NALU_HEADER_LEN + (write_size - header_len);
   			unsigned char *input_buf = Hover_MemAlloc(input_size);
   			memset(input_buf, 0, input_size);
   			input_buf[NALU_STARTER_LEN - 1] = 1; // NaluStarter - [00, 00, 00, 01]
   			memcpy(input_buf + NALU_STARTER_LEN, &nalu_header, NALU_HEADER_LEN); // NaluHeader
   			memcpy(input_buf + NALU_STARTER_LEN + NALU_HEADER_LEN, nalu_payload, write_size - header_len); // NaluPayload
   
   			return h264_buffer_input(input_buf, input_size);
 		}
 		else // Fu包为 Nalu的其他位置，只需要写入 NaluPayload.
 		{
   			int input_size = write_size - header_len;
   			unsigned char *input_buf = Hover_MemAlloc(input_size);
   			memset(input_buf, 0, input_size);
   			memcpy(input_buf, nalu_payload, input_size); // NaluPayload
   
   			return h264_buffer_input(input_buf, input_size);
 		}
   }
   else // 完整 Nalu 包需要写入 NaluStarter + NaluHeader + NaluPayload.
   {
 		int input_size = NALU_STARTER_LEN + (write_size - RTP_HEADER_LEN);
 		unsigned char *input_buf = Hover_MemAlloc(input_size);
 		memset(input_buf, 0, input_size);
 		input_buf[NALU_STARTER_LEN - 1] = 1; // NaluStarter - [00, 00, 00, 01]
 		memcpy(input_buf + NALU_STARTER_LEN, rtp_payload, write_size - RTP_HEADER_LEN); // RtpPayload = NaluHeader + NaluPayload.
 
 		return h264_buffer_input(input_buf, input_size);
   }
}
```


## H.264 RTP包的时间戳计算

**RTP包的时间戳起始值是随机的**，可以由服务端的响应RTSP请求 的`PLAY`方法时，在响应头的RTP-Info字段中指定

```http
RTSP/1.0 200 OK 
CSeq: 5 
Session: 1371694814 
RTP-Info: url=rtsp://10.86.77.14:554/h264/ch1/sub/av_stream/trackID=1;seq=64758;rtptime=2794043652 
```

RTP包的时间戳增量怎么计算？

假设时钟频率为90000，帧率为25

频率为90000表示一秒用90000点来表示

帧率为25，那么一帧就是1/25秒

所以一帧有90000*(1/25)=3600个点来表示

因此每一帧数据的时间增量为3600

```
rtpHeader.timestamp += 90000/fps
sleep(1000*1000/fps)
```

## 组装RTP包中的分片NALU

本人从海康的IP摄像头中抓取了部分报文，客户端使用RTP/AVP/TCP方式传输RTP数据，从发出PLAY请求后，从同一端口接收RTP数据包，每个RTP数据包包括TCP头部和RTP包，经过分析，其RTP载荷中的（I帧）IDR类型的NALU是会分片为几十个RTP包。（SPS、PPS、SEI类型的NALU一般不会出现分片)

通过比较不同帧的RTP分片数据包，一般开始发送数据之前会先发送SPS、PPS、SEI等类型的包，然后IDR帧数据的第一个分片包的第二个字节FU Header最高字节是1，也就是起始标志，最后一个分片的结束标志是1，且RTP头部的Marker为置为1，标志一帧结束，接下来几个RTP包携带的NALU是帧的数据划分片段(两个I帧之间的P帧、B帧等）。

## 解码流程

先理解关键知识点，不知道有没有误导，H264 的帧是以 NAL 单元的单位来传送的，一个 NAL 单元包含一帧（I帧 或 P帧 或 B帧），这三种类型的帧可以百度。所谓的 NAL 单元就是去掉 SPS、PPS 的视频帧， I 帧是关键帧，所有的解析都需要靠它，两个 I 帧之间被称为视频序列，I 帧头部需要加入 SPS 和 PPS，这两个之间需要 0x00000001 来分割， 0x00 0x00 0x00 0x01 + SPS的 Base64 解码形式 + 0x00 0x00 0x00 0x01 + PPS 的解码形式 + 0x00 0x00 0x00 0x01 视频帧（IDR帧） 这样组成的一个 buffer，FFMPEG 的 H264 解码器才能成功解码。

- 一般H264码流最开始的两个NALU是`SPS`和`PPS`，然后`SEI`可选，接下来NALU是`IDR`。

- SPS、PPS、SEI这三种NALU不属于帧的范畴。

```c
typedef enum {
	NALU_TYPE_SLICE    = 1,
	NALU_TYPE_DPA      = 2,
	NALU_TYPE_DPB      = 3,
	NALU_TYPE_DPC      = 4,
	NALU_TYPE_IDR      = 5,
	NALU_TYPE_SEI      = 6,
	NALU_TYPE_SPS      = 7,
	NALU_TYPE_PPS      = 8,
	NALU_TYPE_AUD      = 9,
	NALU_TYPE_EOSEQ    = 10,
	NALU_TYPE_EOSTREAM = 11,
	NALU_TYPE_FILL     = 12,
} NaluType;
```

其中其中BasicUsageEnvironment、groupsock、liveMedia、UsageEnvironment、WindowsAudioInputDevice是基础类库，以静态库的形式存在，testProgs则主要存放一些测试代码，mediaServer则是服务器的主程序，以一个可执行文件形式存在。

UsageEnvironment：UsageEnvironment和TaskScheduler类用于调度延迟事件、给异步读事件分配句柄、输出错误或警告消息。HashTable类定义了通用哈希表接口，该接口被源码的其他部分使用。这些类是抽象类，需要去继承并实现。

groupsock：这个库的类封装了网络接口和套接字，需要注意的是，Groupsock封装了发送或接收主播报文的操作。

liveMedia：这个库定义了以Medium类为根的树状层级类，封装了大量流媒体类型和编码。

BasicUsageEnvironment：这个类是UsageEnvironment的一个实现类。
testProgs：一些测试程序。

WindowsAudioInputDevice：这个库是liveMedia中抽象类AudioInputDevice的实现，能在Windows上从输入设备读取PCM音频采样。

mediaServer：服务器主程序。

proxyServer：代理服务器，以一个可执行文件形式存在。


In [9]:
# 以下数据从 wireshare 中截取复制的大量 RTP包的 载荷部分
rtp_payload = ['674d001e9635416024d37010101400001c2000057e4010', '68ee3c80', '06e501b580',
               '...'
              ]
rtp_payload = [bytes.fromhex(i) for i in rtp_payload]

In [12]:
SEPARATE = bytes([0, 0, 0, 1])


def parse_rtp_payload(rtp_payload: bytes) -> bytes:
    """
    :param rtp_payload: rtp payload
    """
    result = b''
    nalu_header = b''
    nalu_payload = b''

    nalu_type = rtp_payload[0] & 0x1F
    # 最高位必须是 0
    if nalu_type > 128:
        print('first Byte is invalid')
        return result
    # 如果是 rtp分片
    if nalu_type == 28 or nalu_type == 29:
        # 取出 FU_Indicator , FU_Header
        FU_Indicator, FU_Header = rtp_payload[:2]
        nalu_type = FU_Header & 0x1F
        nalu_payload = rtp_payload[2:]

        if FU_Header & 0x80 == 0x80:  # Fu包为 Nalu的起始位置，需要写入 NaluStarter + NaluHeader + NaluPayload.
            print('Srtart slice: ', end=' ')
            nalu_header = (FU_Indicator & 0xE0) | (
                FU_Header & 0x1F)  # FuIndicator 的前3位和 FuHeader的后5位
            result = SEPARATE + bytes([nalu_header]) + nalu_payload

        else:  # Fu包为 Nalu的其他位置，只需要写入 NaluPayload.
            if FU_Header & 0x40 == 0x40:
                print('End slice: ', end=' ')
            else:
                print('Mid slice: ', end=' ')

            result = nalu_payload
    # 不分片，完整 Nalu 包需要写入 NaluStarter + NaluHeader + NaluPayload.
    else: 
        result = SEPARATE + rtp_payload

    if nalu_type == 5:
        print('IDR')
    elif nalu_type == 1:
        print('Slice')
    elif nalu_type == 6:
        print('SEI')
    elif nalu_type == 7:
        print('SPS')
    elif nalu_type == 8:
        print("PPS")
    else:
        print('type {}'.format(nalu_type))
    return result

In [10]:
v = rtp_payload[0]
v.hex()

'674d001e9635416024d37010101400001c2000057e4010'

In [17]:
m = b''
index = 0
for i in rtp_payload[:]:
    print(index, end=' ')
    m = m + parse_rtp_payload(i)
    index += 1
len(rtp_payload)

0 SPS
1 PPS
2 SEI
3 Srtart slice:  IDR
4 Mid slice:  IDR
5 Mid slice:  IDR
6 Mid slice:  IDR
7 Mid slice:  IDR
8 Mid slice:  IDR
9 Mid slice:  IDR
10 Mid slice:  IDR
11 Mid slice:  IDR
12 Mid slice:  IDR
13 Mid slice:  IDR
14 Mid slice:  IDR
15 Mid slice:  IDR
16 Mid slice:  IDR
17 Mid slice:  IDR
18 Mid slice:  IDR
19 Mid slice:  IDR
20 Mid slice:  IDR
21 Mid slice:  IDR
22 Mid slice:  IDR
23 Mid slice:  IDR
24 Mid slice:  IDR
25 Mid slice:  IDR
26 Mid slice:  IDR
27 Mid slice:  IDR
28 Mid slice:  IDR
29 Mid slice:  IDR
30 Mid slice:  IDR
31 Mid slice:  IDR
32 Mid slice:  IDR
33 Mid slice:  IDR
34 Mid slice:  IDR
35 Mid slice:  IDR
36 Mid slice:  IDR
37 Mid slice:  IDR
38 Mid slice:  IDR
39 Mid slice:  IDR
40 Mid slice:  IDR
41 Mid slice:  IDR
42 Mid slice:  IDR
43 Mid slice:  IDR
44 Mid slice:  IDR
45 Mid slice:  IDR
46 End slice:  IDR
47 Slice
48 Slice
49 Slice
50 Slice
51 Slice
52 Slice
53 Srtart slice:  Slice
54 End slice:  Slice
55 Srtart slice:  Slice
56 End slice:  Slice
57 Srt

60

In [14]:
with open(r'C:\Users\13438\Desktop\rtp_h264.h264', 'wb') as f:
    f.write(m)

In [4]:
a = '7c05c8cb7ef1ec79674331e90672ff67ee3672f15537bb980d466663f022e29d7919a8c8a81b163e39edaf0b19bf8bcef2f03d419c35b4a6426abb2a6938d4d2ec7954b432ab9e729deb3fab4ec25099a366533d3bacfaf9bd33050f9bb12b4d6cfdbf6e11676e4a7bdd033320e80c99cd44c82b8ef1967bf6fb54f7c5cdbb904b9ea3f2e14e362dcf8df0d5cec0b77925d2842a74751ab2e66b620709256d39ea185e7f386cc7ea5d261194a05f86bd8b7cbcac513a67e5c2f1a096ea34f8847f7922e2cc153d59fe90c633e90c8791be9fd893a8faedc6bc2ad47875e7c05fac34e342d6949d4d8171e32594e4a3d913efc3acc71f3db5cf47ddbc00ea5f7ade3fa656915dee3c800cf40d7d965ad749e83a4765114326b2e030d109d68cf381a705993a5ad4657c2d4f7db05bb7c5131635a4ffce4ab67a0a0dc2e06a4b329e04c8f2bfe8712bbde9f0c6ff42ecb8279a74a8ae756b6ef42fe0b07013e715bd8e0d749bc55f8f5d075b6dbdfdd4d14f17bd54152dccd3b39d67ae77dbc86f22ed64a226d1a27c45cfd90d9f8bac15bc4267c76baf5e1525b3ef8a5bdbfbfd864404f620357bb4ed462921202e07b85d720e12f723aa8dd1099feb3dab05207d1e0c1df6d68d6c5b62e39eb2aa374a4b7d06e26d5a33ab90e1b681111691b554684fffeb30a65d7a0b2b244e0cc06b4385dec98e8d6fd0a800cf9a3aa5268ea12c575672e89e2acc3f81fd9ac714bb0b30b8539580c6bfe8faadcafe1e530052a1e82f984bc67cc776cdcecbd08d8848db316afb10dc0e7224aec8f2ec37537ebaf47882564025b845a6c2142ad65d51904dbb0287ca231aeeefb9e8a274c0201dc62786643a7a8c3ee00711a87d1edc6cf82d0ad0c4073dbc49682a623758ba9b6919cdda47c691c91328aea4b5b50584f56525be9e981cb578bfee3b125d8ca262da13fdba3f844697bb9a09a1745e45cd74690519c83e6a6adef96cf07e56af542a6c37474bb553fa10909186477666367b399c44b45f62ec2f743418c1daf00156e92bfffea6f8bdd9f1d89b40b8f74f0d58dc81b49de4deae631fac6d123176352ed3a86cb2dc268c5ad6e61f637535d4e5dcdd543785492221d7914be5e9dc3e28cb70f0f4dbed7d1d7833322c0738b4523c02712ad2c460af5299141738485a33854d3d90b6616c3fcc612f6688881fd5fca47d2f95a5aea5aee000af1c1dbeeb9458d639b1addf8cb5cb7eae6820f4ad81a234636d79157837cc965754d6c5ed3579ef84f1067eb1fd17402a7765b497dcbbb8e71b3b5ef738c45ebc4d046f647f481cf1a9383e230c3a406be971df5f4dc13b62da0d78b94a5519743bc5553758c6b43ea3167a3c7cc614de30ccf6fc5ce90e850d9fe510ffbefde98f27de7d0dfda05b1f538462cbbb468c2b61236253aed3d66295bfd96d682984df50d07ea87066e81cd8fe6f3372ac47520e9856f6fcb9c775937a6b35c4158258586670a22b74b38639fff644aeca94fc22e4f33ef3d37f3524acd0c5317f12744251c3377f39ad5da5b9d4dfcd4bf754e2c864a5609059934759fa591d36c8cb532b5705bf7a36ab2b4ace9b384df207b295ce961cd600bb62c429fba3a4a6b050591a1a188eb51a2d307e8ef75f41769a6bac01302969aa1e2c1e84a026e5342685e9011b8363b447e3965adccf015a06a9897779b07bd6d198e84f52a48633f25f8a5662b4b37febbbeeb3c39675c4202153d3dcd75809354fb359b45be714c1bc638176e5ec59bc3b6892c5a76795990fcf8005b51562f8fefa059fcb1b11996252cf97b2fb7b752cf2046b1b82b32909ea93460662e82c143a0ff921eee66c9f1d55a1971cb59b11d124a137414d9dc7050aba44046dbb574ee7e18635f1839a6f9b43c60fd88f05ab868221b6ccd8334f49c138a3672248880b7eef89a0efded1840da6d3188ec5db45b5f562cad09bf2891a79077ceafafbf99a426eee86fcdd7d3c2cf8459f870e10b4fda4a714df'

b = bytes.fromhex(a)

In [5]:
c = [print(hex(i)[2:].zfill(2), bin(i)[2:].zfill(8)) for i in b]

7c 01111100
05 00000101
c8 11001000
cb 11001011
7e 01111110
f1 11110001
ec 11101100
79 01111001
67 01100111
43 01000011
31 00110001
e9 11101001
06 00000110
72 01110010
ff 11111111
67 01100111
ee 11101110
36 00110110
72 01110010
f1 11110001
55 01010101
37 00110111
bb 10111011
98 10011000
0d 00001101
46 01000110
66 01100110
63 01100011
f0 11110000
22 00100010
e2 11100010
9d 10011101
79 01111001
19 00011001
a8 10101000
c8 11001000
a8 10101000
1b 00011011
16 00010110
3e 00111110
39 00111001
ed 11101101
af 10101111
0b 00001011
19 00011001
bf 10111111
8b 10001011
ce 11001110
f2 11110010
f0 11110000
3d 00111101
41 01000001
9c 10011100
35 00110101
b4 10110100
a6 10100110
42 01000010
6a 01101010
bb 10111011
2a 00101010
69 01101001
38 00111000
d4 11010100
d2 11010010
ec 11101100
79 01111001
54 01010100
b4 10110100
32 00110010
ab 10101011
9e 10011110
72 01110010
9d 10011101
eb 11101011
3f 00111111
ab 10101011
4e 01001110
c2 11000010
50 01010000
99 10011001
a3 10100011
66 01100110
53 01010011
3d 0

35 00110101
79 01111001
ef 11101111
84 10000100
f1 11110001
06 00000110
7e 01111110
b1 10110001
fd 11111101
17 00010111
40 01000000
2a 00101010
77 01110111
65 01100101
b4 10110100
97 10010111
dc 11011100
bb 10111011
b8 10111000
e7 11100111
1b 00011011
3b 00111011
5e 01011110
f7 11110111
38 00111000
c4 11000100
5e 01011110
bc 10111100
4d 01001101
04 00000100
6f 01101111
64 01100100
7f 01111111
48 01001000
1c 00011100
f1 11110001
a9 10101001
38 00111000
3e 00111110
23 00100011
0c 00001100
3a 00111010
40 01000000
6b 01101011
e9 11101001
71 01110001
df 11011111
5f 01011111
4d 01001101
c1 11000001
3b 00111011
62 01100010
da 11011010
0d 00001101
78 01111000
b9 10111001
4a 01001010
55 01010101
19 00011001
74 01110100
3b 00111011
c5 11000101
55 01010101
37 00110111
58 01011000
c6 11000110
b4 10110100
3e 00111110
a3 10100011
16 00010110
7a 01111010
3c 00111100
7c 01111100
c6 11000110
14 00010100
de 11011110
30 00110000
cc 11001100
f6 11110110
fc 11111100
5c 01011100
e9 11101001
0e 00001110
85 1

In [16]:
parse_rtp_payload(rtp_payload[0])

SPS


b'\x00\x00\x00\x01gM\x00\x1e\x965A`$\xd3p\x10\x10\x14\x00\x00\x1c \x00\x05~@\x10'